In [1]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
import numpy as np
from scipy.optimize import minimize

def solve_hotelling3(prices, cost):
    n = len(prices)
    arc = 1.0 / n
    s = np.zeros(n)
    for i in range(n):
        left = (i-1)%n
        right = (i+1)%n
        s[i] = arc + (prices[right] - 2*prices[i] + prices[left])/(2*cost)
    s = np.maximum(s, 0)   # optional, for numerical safety
    s /= s.sum()
    return s

def merged_profits(phi, lambd, merged_idxs=(1,2), wtp=25, mc=None):
    phi = np.array(phi)
    n = len(phi)
    a, b = merged_idxs
    c = [i for i in range(n) if i not in merged_idxs][0]

    def make_prices(p_merged, p_rival):
        prices = np.zeros(n)
        prices[a] = prices[b] = p_merged
        prices[c] = p_rival
        return prices

    def merged_objective(p_merged, p_rival):
        if (p_merged < phi[a]) or (p_merged > wtp):
            return 10  # Penalize
        if (p_rival < phi[c]) or (p_rival > wtp):
            return 10  # (really only relevant for two-dim minimize, but harmless)
        prices = make_prices(p_merged, p_rival)
        s = solve_hotelling3(prices, lambd)
        return -((prices[a] - phi[a]) * s[a] + (prices[b] - phi[b]) * s[b])
    
    def rival_objective(p_rival, p_merged):
        if (p_rival < phi[c]) or (p_rival > wtp):
            return 10  # Penalize
        if (p_merged < phi[a]) or (p_merged > wtp):
            return 10
        prices = make_prices(p_merged, p_rival)
        s = solve_hotelling3(prices, lambd)
        return -((prices[c] - phi[c]) * s[c])

    # Initial guess
    price_eq = phi + lambd
    p_merged = price_eq[a]
    p_rival = price_eq[c]

    for _ in range(100):
        p_merged_old = p_merged
        p_rival_old = p_rival

        res_m = minimize(lambda x: merged_objective(x[0], p_rival), [p_merged], method='Nelder-Mead', options={'disp': False})
        p_merged = res_m.x[0]  # No need to clip, penalty ensures feasible region
        res_c = minimize(lambda x: rival_objective(x[0], p_merged), [p_rival], method='Nelder-Mead', options={'disp': False})
        p_rival = res_c.x[0]

        if abs(p_merged - p_merged_old) < 1e-8 and abs(p_rival - p_rival_old) < 1e-8:
            break

    prices = make_prices(p_merged, p_rival)
    s = solve_hotelling3(prices, lambd)
    insurer_profits = (prices - phi) * s
    hospital_profit = ( phi * s ).sum()
    return hospital_profit, insurer_profits, s, prices

# Example usage
phi = np.array([10.95692717, 9.46563451, 9.46563451])
lambd = 6
wtp = 25
hospital_profit, insurer_profits, shares, prices = merged_profits(phi, lambd, merged_idxs=(1,2), wtp=wtp)
print("hospital profit:", hospital_profit)
print("insurer profits:", insurer_profits)
print("shares:", shares)
print("prices:", prices)

hospital profit: 10.004881477451848
insurer profits: [0.78452954 1.22270091 1.22270091]
shares: [0.36159701 0.31920149 0.31920149]
prices: [13.12655114 13.2961332  13.2961332 ]


In [3]:
merged_profits([23,25,25],6)

(23.66666666666667,
 array([4., 1., 1.]),
 array([0.66666667, 0.16666667, 0.16666667]),
 array([29., 31., 31.]))

In [4]:
def solve_hotelling2(phi, lambd, idx):
    i, j = idx
    p_i = (2 * phi[i] + phi[j]) / 3 + lambd
    p_j = (2 * phi[j] + phi[i]) / 3 + lambd
    prices = np.zeros(len(phi))
    prices[i] = p_i
    prices[j] = p_j
    return prices


def calc_price(phi, lambd, wtp):
    phi = np.array(phi)
    n = len(phi)
    price = np.zeros(n)
    eps = 1e-8

    if n == 1:
        price = np.array([max(phi[0], min(wtp, wtp-lambd))])
    elif n == 2:
        price = solve_hotelling2(phi, lambd, [0, 1])
    elif n == 3:
        # All equal? Symmetric solution
        if np.max(phi) - np.min(phi) < eps:
            price = np.ones(n) * (phi[0] + lambd/3)
        else:
            # Try general solution
            price_try = solve_hotelling3(phi, lambd)
            ok = (price_try >= phi - eps) & (price_try <= wtp + eps)
            if np.all(ok):
                price = price_try
            else:
                # Find two lowest-cost firms, use 2-firm solution
                active = np.argsort(phi)[:2]
                price = solve_hotelling2(phi, lambd, active)
    else:
        raise NotImplementedError("Only supports n=1, 2, or 3.")
    return price




def shares_hotelling2(prices, cost, idx):
    # idx: indices of two active firms
    i, j = idx
    p_i, p_j = prices[i], prices[j]
    s_i = 0.5 + (p_j - p_i) / (2 * cost)
    s_j = 1.0 - s_i
    s = np.zeros(len(prices))
    s[i], s[j] = s_i, s_j
    return s
    
    
def calc_s(phi, cost, wtp, circle=True):
    phi = np.array(phi)
    n = len(phi)
    prices = calc_price(phi, cost, wtp)

    if np.all(prices == 0):
        return np.zeros(n)

    active = np.where(prices > 0)[0]

    if len(active) == 1:
        s = np.zeros(n)
        s[active[0]] = 1.0
        return s
    elif len(active) == 2:
        return shares_hotelling2(prices, cost, active)
    elif len(active) == 3:
        # All active; circle
        s = solve_hotelling3(prices, cost)
        # If crazy numbers (e.g., shares < 0), fallback to zeros.
        if np.any(s < 0) or np.any(s > 1):
            return np.zeros(n)
        else:
            return s
    else:
        return np.zeros(n)


def calc_profits(phi, cost, wtp, mc):
    s = calc_s(phi, cost, wtp)
    prices = calc_price(phi, cost, wtp)
    hospital_profit = np.sum(s * (phi - mc))
    insurer_profits = s * (prices - phi)
    return hospital_profit, insurer_profits, s, prices


def analytic_phi(cost, wtp, n, belief="active"):
    if n==0:
        return 0
    if n == 1:
        return (wtp - cost) / 2
    elif n == 2:
        if belief == "active":
            return 0.75*cost + (wtp-cost)/2
        elif belief == "passive":
            return 1.5*cost
    elif n == 3:
        if belief == "active":
            return cost + wtp/2
        elif belief == "passive":
            return 9/4*cost
    return None


def nash_obj_helper(phi_vec, cost, wtp, mc, betas, disagreement=0, merged_idxs=None):
    # Default: use standard calc_profits
    hospital_profit, ins, s, prices = calc_profits(phi_vec, cost, wtp, mc)
    profit_idxs = [0]

    # If merging, and first agent is in the merged group, use merged profits
    if merged_idxs is not None: #TODO not sure if 0 not in merged_idxs
        hosp, ins, s, prices = merged_profits(phi_vec, cost, merged_idxs, wtp, mc)
        if 0 in merged_idxs:
            profit_idxs = merged_idxs
        
    ins_profit = np.sum(ins[profit_idxs])
    ins_beta = np.mean([betas[i] for i in profit_idxs])
    # Defensive: avoid log(0) or negative argument
    if hospital_profit - disagreement <= 1e-8 or ins_profit <= 1e-8 or hospital_profit <= 1e-8 :
        return 10
    #print('hosp profit, disagreement, ins_profit', hospital_profit,disagreement, ins_profit)
    obj = np.log(hospital_profit - disagreement) * (1 - ins_beta) + np.log(ins_profit) * ins_beta
    return -obj


#TODO nash objective actually depends on the thing we're optimzing over e.g. dphi1/dphi2 = 0... 
def passive_disagreement(phi_vec, cost, wtp, mc, merged_idxs=None):
    s = calc_s(phi_vec, cost, wtp)
    phi = np.array(phi_vec)
    mc = np.array(mc)
    if merged_idxs is None:
        return np.sum(s * (phi - mc)) - s[0] * (phi[0] - mc[0])

    merged_idxs = list(merged_idxs)
    other_idxs = [i for i in range(len(phi)) if i not in merged_idxs]
    return np.sum([s[i] * (phi[i] - mc[i]) for i in other_idxs])


def recursive_disagreement(phi_vec, cost, wtp, mc, betas, analytic_base=True, merged_idxs=None):
    n = len(phi_vec)
    if analytic_base and n <= 3 and merged_idxs is None:
        # For 1- or 2-firm base: use all analytic phi (active beliefs by default here)
        sub_phi = np.array([analytic_phi(cost, wtp, n-1, belief='active')] * n)
        hospital_profit, _, _, _ = calc_profits(sub_phi, cost, wtp, mc[:n])
        return hospital_profit
    
    # basically use the logic from before " merged_idxs is not None and 0 in merged_idxs" to figure out if merged insurer
    #if  merged_idxs is not None and 0 in merged_idxs then v-lambda/2 o.w.  v-lambda
    if analytic_base and n <= 3 and merged_idxs is not None:
        if 0 in merged_idxs:
            return (wtp - cost) / 2
        else:
            return (wtp - cost/2) /2
    if n == 1:
        # (should never reach here if analytic_base is True... but as fallback)
        phi_init = [analytic_phi(cost, wtp, 1, belief='active')]
        result = minimize(lambda phi: nash_obj(phi, cost, wtp, [mc[0]], [betas[0]]),
                          phi_init, method='Nelder-Mead', options={'disp': False})
        phi_star = result.x
        hosp_profit, _, _, _ = calc_profits(phi_star, cost, wtp, [mc[0]])
        return hosp_profit
    # n-1 agents: recursively solve Nash for subgame (all but the first)
    keep = list(range(1, n))
    phi_sub = np.array([phi_vec[j] for j in keep])
    mc_sub = np.array([mc[j] for j in keep])
    betas_sub = [betas[j] for j in keep]
    phi_sub_sol = solve_nash(phi_sub, cost, wtp, mc_sub, betas_sub, analytic_base=analytic_base)
    hospital_profit, _, _, _ = calc_profits(phi_sub_sol, cost, wtp, mc_sub)
    return hospital_profit


def nash_obj(phi_vec, cost, wtp, mc, betas, active=True, analytic_base=True, 
             phi_tmp_prev=0, merged_idxs=None):
    disagreement = 0
    n = len(phi_vec)
    if n == 1:
        return disagreement
    if active:
        disagreement = recursive_disagreement(phi_vec, cost, wtp, mc, betas, analytic_base=analytic_base, 
                                              merged_idxs=merged_idxs)
        #print(phi_vec,disagreement,merged_idxs )
    else:
        phi_vec_tmp  = phi_vec.copy()
        phi_vec_tmp[0] = phi_tmp_prev
        disagreement = passive_disagreement(phi_vec_tmp, cost, wtp, mc, merged_idxs=merged_idxs)
        
    return nash_obj_helper(phi_vec, cost, wtp, mc, betas, disagreement, merged_idxs)



#This is the work in progress one...
def solve_nash(phi_init, cost, wtp, mc, betas=None, active=False, maxiter=25, tol=1e-7, 
               analytic_base=True, merged_idxs=None):
    n = len(phi_init)
    if betas is None:
        betas = [0.5] * n
    phi = np.array(phi_init, dtype=float)
    phi_prev = phi.copy()

    # 1. Determine update indices
    if merged_idxs is not None:
        merged_idxs = sorted(list(merged_idxs))
        rep_idx = merged_idxs[0]  # Use the lowest index as "representative"
        update_indices = [rep_idx] + [i for i in range(n) if i not in merged_idxs]
    else:
        update_indices = list(range(n))

    for it in range(maxiter):
        phi_prev[:] = phi[:]  # Save old for diff check

        for k in update_indices:
            
            merged_idxs_rolled = None
            if merged_idxs is not None:
                merged_idxs_rolled = [((idx - k) % n) for idx in merged_idxs]

            
            
            def helper(phi_k_scalar):
                phi_tmp = phi_prev.copy()
                phi_tmp_prev = phi_tmp[0]
                phi_tmp[k] = phi_k_scalar[0]
                # 2a. When updating merged block, force phis in group equal (in phi_tmp)
                if merged_idxs is not None and k == rep_idx:
                    for j in merged_idxs:
                        phi_tmp[j] = phi_k_scalar[0]
                # Roll so agent k is in 0 position for nash_obj
                phi_tmp_roll = np.roll(phi_tmp, -k)
                mc_roll = np.roll(mc, -k)
                betas_roll = np.roll(betas, -k)

                if active:
                    res = nash_obj(phi_tmp_roll, cost, wtp, mc_roll, betas_roll, active=True,
                                    merged_idxs=merged_idxs_rolled)
                    #print('obj active',merged_idxs_rolled, res)
                    return res
                else:
                    return nash_obj(phi_tmp_roll, cost, wtp, mc_roll, betas_roll, active=False,
                                    phi_tmp_prev=phi_tmp_prev,
                                    merged_idxs=merged_idxs_rolled)
            res = minimize(helper, [phi[k]], method='Nelder-Mead', options={'disp': False})
            phi[k] = res.x[0]
            # 2b. Synchronize all merged phis on representative updates
            if merged_idxs is not None and k == rep_idx:
                for j in merged_idxs:
                    phi[j] = phi[k]
        diff = np.max(np.abs(phi - phi_prev))
        phi_prev = phi.copy()
        #print('yo 2, ', phi_prev)

    return phi


In [5]:
# --- Model run/prints ---
COST = 6
WTP = 25
MC = np.array([0, 0, 0])
betas = [0.5, 0.5, 0.5]
betas2 = [0.5, 0.5]

n =3
phi_init_pass = [analytic_phi(COST, WTP, n, belief='passive')]*n
phi_init_act = [1.5*analytic_phi(COST, WTP, n, belief='active')]*n

In [6]:
# Example input (adapt as needed)

COST = 6
WTP = 25
MC = np.zeros(3)
betas = None # (if you use these elsewhere, ignore in merged_profits)
merged_idxs = (1,2)



# --- ACTIVE BELIEFS SCENARIO ---
phi_active_merged = solve_nash(phi_init_act, COST, WTP, MC, betas, active=True, merged_idxs=merged_idxs)
print("\nWITH MERGER (1,2), ACTIVE BELIEFS:")
print("  ACTIVE φ:", phi_active_merged)
hospital_profit, insurer_profits, s, prices = merged_profits(phi_active_merged, COST, merged_idxs=merged_idxs, wtp=WTP)
print("  hospital profit:", hospital_profit)
print("  insurer profits:", insurer_profits)
print("  market shares:", s)
print("  prices:", prices)


WITH MERGER (1,2), ACTIVE BELIEFS:
  ACTIVE φ: [12.97685237 12.96966961 12.96966961]
  hospital profit: 12.972859075568767
  insurer profits: [1.18305453 0.92725649 0.92725649]
  market shares: [0.44404416 0.27797792 0.27797792]
  prices: [15.6411246  16.30538958 16.30538958]


In [7]:
# --- PASSIVE BELIEFS SCENARIO ---
phi_passive_merged = solve_nash(phi_init_pass, COST, WTP, MC, betas, active=False, merged_idxs=merged_idxs)
print("\nWITH MERGER (1,2), PASSIVE BELIEFS:")
print("  PASSIVE φ:", phi_passive_merged)
hospital_profit, insurer_profits, s, prices = merged_profits(phi_passive_merged, COST, merged_idxs=merged_idxs, wtp=WTP)
print("  hospital profit:", hospital_profit)
print("  insurer profits:", insurer_profits)
print("  market shares:", s)
print("  prices:", prices)




WITH MERGER (1,2), PASSIVE BELIEFS:
  PASSIVE φ: [4.09708551 4.55425296 4.55425296]
  hospital profit: 4.339455671969977
  insurer profits: [1.32455064 0.8432201  0.8432201 ]
  market shares: [0.46984379 0.2650781  0.2650781 ]
  prices: [6.91621529 7.73527806 7.73527806]


In [8]:
def run_active_passive_tables_merger():
    import numpy as np
    import pandas as pd

    v_vals = np.arange(24, 28)  # 24 to 27 inclusive
    lambdas = np.arange(4, 8)   # 4 to 7 inclusive
    n = 3
    MC = np.array([0, 0, 0])
    betas = [0.5]*n

    def make_headers(vary_label, phi_label):
        return [
            vary_label, 
            f"{phi_label} (merged)", f"{phi_label} (other)", 
            "hospital_profit",
            "insurer_profits_merged", "insurer_profits_other",
            "shares_merged", "shares_other",
            "prices_merged", "prices_other",
        ]

    table_active_v, table_passive_v = [], []
    table_active_l, table_passive_l = [], []

    # Table 1: Vary v, fixed lambda=5
    l_fixed = 5
    merged_idx, other_idx = 0, 2  # update if your merged_profits output is different

    for v in v_vals:
        print(v)
        # ACTIVE (phi: reimbursement)
        phi_init_act = [analytic_phi(l_fixed, v, n, belief='active')]*n
        phi_active = solve_nash(phi_init_act, l_fixed, v, MC, betas, active=True, merged_idxs=(0,1))
        hospital_profit_a, insurer_profits_a, s_a, prices_a = merged_profits(
            phi_active, l_fixed, merged_idxs=(0,1), wtp=v, mc=MC)
        row_active = [
            v, 
            np.round(phi_active[merged_idx],2), np.round(phi_active[other_idx],2),
            np.round(hospital_profit_a,2),
            np.round(insurer_profits_a[merged_idx],2), np.round(insurer_profits_a[other_idx],2),
            np.round(s_a[merged_idx],2), np.round(s_a[other_idx],2),
            np.round(prices_a[merged_idx],2), np.round(prices_a[other_idx],2)
        ]
        table_active_v.append(row_active)

        # PASSIVE
        phi_init_pass = [analytic_phi(l_fixed, v, n, belief='passive')]*n
        phi_passive = solve_nash(phi_init_pass, l_fixed, v, MC, betas, active=False, merged_idxs=(0,1))
        hospital_profit, insurer_profits, s, prices = merged_profits(
            phi_passive, l_fixed, merged_idxs=(0,1), wtp=v, mc=MC)
        row_passive = [
            v, 
            np.round(phi_passive[merged_idx],2), np.round(phi_passive[other_idx],2),
            np.round(hospital_profit,2),
            np.round(insurer_profits[merged_idx],2), np.round(insurer_profits[other_idx],2),
            np.round(s[merged_idx],2), np.round(s[other_idx],2),
            np.round(prices[merged_idx],2), np.round(prices[other_idx],2)
        ]
        table_passive_v.append(row_passive)

    # Table 2: Vary lambda, fixed v=25
    v_fixed = 25
    for lmbd in lambdas:
        print(lmbd)
        # ACTIVE
        phi_init_act = [analytic_phi(lmbd, v_fixed, n, belief='active')]*n
        phi_active = solve_nash(phi_init_act, lmbd, v_fixed, MC, betas, active=True, merged_idxs=(0,1))
        hospital_profit_a, insurer_profits_a, s_a, prices_a = merged_profits(
            phi_active, lmbd, merged_idxs=(0,1), wtp=v_fixed, mc=MC)
        row_active = [
            lmbd,
            np.round(phi_active[merged_idx],2), np.round(phi_active[other_idx],2),
            np.round(hospital_profit_a,2),
            np.round(insurer_profits_a[merged_idx],2), np.round(insurer_profits_a[other_idx],2),
            np.round(s_a[merged_idx],2), np.round(s_a[other_idx],2),
            np.round(prices_a[merged_idx],2), np.round(prices_a[other_idx],2)
        ]
        table_active_l.append(row_active)

        # PASSIVE
        phi_init_pass = [analytic_phi(lmbd, v_fixed, n, belief='passive')]*n
        phi_passive = solve_nash(phi_init_pass, lmbd, v_fixed, MC, betas, active=False, merged_idxs=(0,1))
        hospital_profit, insurer_profits, s, prices = merged_profits(
            phi_passive, lmbd, merged_idxs=(0,1), wtp=v_fixed, mc=MC)
        row_passive = [
            lmbd,
            np.round(phi_passive[merged_idx],2), np.round(phi_passive[other_idx],2),
            np.round(hospital_profit,2),
            np.round(insurer_profits[merged_idx],2), np.round(insurer_profits[other_idx],2),
            np.round(s[merged_idx],2), np.round(s[other_idx],2),
            np.round(prices[merged_idx],2), np.round(prices[other_idx],2)
        ]
        table_passive_l.append(row_passive)

    # HEADER setup
    df1 = pd.DataFrame(table_active_v, columns=make_headers("WTP (v)", "phi (active)"))
    df2 = pd.DataFrame(table_passive_v, columns=make_headers("WTP (v)", "phi (passive)"))
    df3 = pd.DataFrame(table_active_l, columns=make_headers("lambda", "phi (active)"))
    df4 = pd.DataFrame(table_passive_l, columns=make_headers("lambda", "phi (passive)"))

    with pd.ExcelWriter('hotelling_results_merger_01.xlsx') as writer:
        df1.to_excel(writer, sheet_name="Active_WTP", index=False)
        df2.to_excel(writer, sheet_name="Passive_WTP", index=False)
        df3.to_excel(writer, sheet_name="Active_lambda", index=False)
        df4.to_excel(writer, sheet_name="Passive_lambda", index=False)

    print("Results saved to hotelling_results_merger_01.xlsx")
    print(df1.head(1))
    print(df2.head(1))
    print(df3.head(1))
    print(df4.head(1))
    return df1, df2, df3, df4

# Run
run_active_passive_tables_merger()

24
25
26
27
4
5
6
7
Results saved to hotelling_results_merger_01.xlsx
   WTP (v)  phi (active) (merged)  phi (active) (other)  hospital_profit  \
0       24                  12.38                 12.38            12.38   

   insurer_profits_merged  insurer_profits_other  shares_merged  shares_other  \
0                    0.77                   0.99           0.28          0.44   

   prices_merged  prices_other  
0          15.16          14.6  
   WTP (v)  phi (passive) (merged)  phi (passive) (other)  hospital_profit  \
0       24                    3.55                    2.7             3.12   

   insurer_profits_merged  insurer_profits_other  shares_merged  shares_other  \
0                    0.62                   1.26           0.25           0.5   

   prices_merged  prices_other  
0           6.04           5.2  
   lambda  phi (active) (merged)  phi (active) (other)  hospital_profit  \
0       4                  12.82                 12.82            12.82   

   insurer_

(   WTP (v)  phi (active) (merged)  phi (active) (other)  hospital_profit  \
 0       24                  12.38                 12.38            12.38   
 1       25                  12.88                 12.88            12.88   
 2       26                  13.37                 13.38            13.37   
 3       27                  13.88                 13.88            13.88   
 
    insurer_profits_merged  insurer_profits_other  shares_merged  shares_other  \
 0                    0.77                   0.99           0.28          0.44   
 1                    0.77                   0.99           0.28          0.44   
 2                    0.77                   0.99           0.28          0.44   
 3                    0.77                   0.99           0.28          0.44   
 
    prices_merged  prices_other  
 0          15.16         14.60  
 1          15.66         15.11  
 2          16.15         15.60  
 3          16.66         16.11  ,
    WTP (v)  phi (passive) (me

In [9]:
#TODO modify these tables to work with merged firm example...

def run_active_passive_tables():
    import numpy as np
    import pandas as pd

    COST = 5
    l_fixed = 5
    v_vals = np.arange(24, 28)  # 24 to 28 inclusive
    n = 3
    MC = np.array([0, 0, 0])
    betas = [0.5]*n

    # --- Table 1: Vary v, fix λ=5 ---
    headers = ["WTP (v)", "phi (active)", "hospital_profit_a", "insurer_profits_a", "shares_a", "prices_a"]
    table_active_v = []

    headers2 = ["WTP (v)", "phi (passive)", "hospital_profit", "insurer_profits", "shares", "prices"]
    table_passive_v = []

    for v in v_vals:
        phi_init_act = [analytic_phi(l_fixed, v, n, belief='active')]*n
        phi_active = solve_nash(phi_init_act, l_fixed, v, MC, betas, active=True)
        hospital_profit_a, insurer_profits_a, s_a, prices_a = calc_profits(phi_active, l_fixed, v, MC)
        # Take only first value from lists:
        row = [v, 
               np.round(phi_active[0],2),
               np.round(hospital_profit_a,2),
               np.round(insurer_profits_a[0],2),   # <-- only first item
               np.round(s_a[0],2),                 # <-- only first item
               np.round(prices_a[0],2)]            # <-- only first item
        table_active_v.append(row)

        phi_init_pass = [analytic_phi(l_fixed, v, n, belief='passive')]*n
        phi_passive = solve_nash(phi_init_pass, l_fixed, v, MC, betas, active=False)
        hospital_profit, insurer_profits, s, prices = calc_profits(phi_passive, l_fixed, v, MC)
        row2 = [v,
                np.round(phi_passive[0],2),
                np.round(hospital_profit,2),
                np.round(insurer_profits[0],2),    # <-- only first item
                np.round(s[0],2),                  # <-- only first item
                np.round(prices[0],2)]             # <-- only first item
        table_passive_v.append(row2)

    # --- Table 2: Vary λ, fix v=25 ---
    lambdas = np.arange(4, 8)  # 4 to 8 inclusive

    headers3 = ["COST (λ)", "phi (active)", "hospital_profit_a", "insurer_profits_a", "shares_a", "prices_a"]
    table_active_l = []

    headers4 = ["COST (λ)", "phi (passive)", "hospital_profit", "insurer_profits", "shares", "prices"]
    table_passive_l = []

    for lmbd in lambdas:
        phi_init_act = [analytic_phi(lmbd, 25, n, belief='active')]*n
        phi_active = solve_nash(phi_init_act, lmbd, 25, MC, betas, active=True)
        hospital_profit_a, insurer_profits_a, s_a, prices_a = calc_profits(phi_active, lmbd, 25, MC)
        row = [lmbd,
               np.round(phi_active[0],2),
               np.round(hospital_profit_a,2),
               np.round(insurer_profits_a[0],2),   # <-- only first item
               np.round(s_a[0],2),                 # <-- only first item
               np.round(prices_a[0],2)]            # <-- only first item
        table_active_l.append(row)

        phi_init_pass = [analytic_phi(lmbd, 25, n, belief='passive')]*n
        phi_passive = solve_nash(phi_init_pass, lmbd, 25, MC, betas, active=False)
        hospital_profit, insurer_profits, s, prices = calc_profits(phi_passive, lmbd, 25, MC)
        row2 = [lmbd,
                np.round(phi_passive[0],2),
                np.round(hospital_profit,2),
                np.round(insurer_profits[0],2),    # <-- only first item
                np.round(s[0],2),                  # <-- only first item
                np.round(prices[0],2)]             # <-- only first item
        table_passive_l.append(row2)

    # --- PRINT TABLES ---
    df1 = pd.DataFrame(table_active_v, columns=headers)
    df2 = pd.DataFrame(table_passive_v, columns=headers2)
    df3 = pd.DataFrame(table_active_l, columns=headers3)
    df4 = pd.DataFrame(table_passive_l, columns=headers4)

    with pd.ExcelWriter('hotelling_results.xlsx') as writer:
        df1_sheet = "Active_WTP"
        df2_sheet = "Passive_WTP"
        df3_sheet = "Active_COST"
        df4_sheet = "Passive_COST"

        df1_title = pd.DataFrame([["=== ACTIVE: Varying WTP (v) with λ=5 ==="]])
        df1_title.to_excel(writer, sheet_name=df1_sheet, header=False, index=False, startrow=0)
        df1.to_excel(writer, sheet_name=df1_sheet, index=False, startrow=2)

        df2_title = pd.DataFrame([["=== PASSIVE: Varying WTP (v) with λ=5 ==="]])
        df2_title.to_excel(writer, sheet_name=df2_sheet, header=False, index=False, startrow=0)
        df2.to_excel(writer, sheet_name=df2_sheet, index=False, startrow=2)

        df3_title = pd.DataFrame([["=== ACTIVE: Varying COST (λ) with v=25 ==="]])
        df3_title.to_excel(writer, sheet_name=df3_sheet, header=False, index=False, startrow=0)
        df3.to_excel(writer, sheet_name=df3_sheet, index=False, startrow=2)

        df4_title = pd.DataFrame([["=== PASSIVE: Varying COST (λ) with v=25 ==="]])
        df4_title.to_excel(writer, sheet_name=df4_sheet, header=False, index=False, startrow=0)
        df4.to_excel(writer, sheet_name=df4_sheet, index=False, startrow=2)

    print("Results saved to hotelling_results_test.xlsx")

    # Only print first row of each DataFrame (as requested)
    print(df1.head(1))
    print(df2.head(1))
    print(df3.head(1))
    print(df4.head(1))

    return df1,df2,df3,df4
# Run
run_active_passive_tables()

Results saved to hotelling_results_test.xlsx
   WTP (v)  phi (active)  hospital_profit_a  insurer_profits_a  shares_a  \
0       24          17.0               17.0               0.56      0.33   

   prices_a  
0     18.66  
   WTP (v)  phi (passive)  hospital_profit  insurer_profits  shares  prices
0       24          11.25            11.25             0.56    0.33   12.91
   COST (λ)  phi (active)  hospital_profit_a  insurer_profits_a  shares_a  \
0         4          16.5               16.5               0.44      0.33   

   prices_a  
0     17.83  
   COST (λ)  phi (passive)  hospital_profit  insurer_profits  shares  prices
0         4            9.0              9.0             0.44    0.33   10.33


(   WTP (v)  phi (active)  hospital_profit_a  insurer_profits_a  shares_a  \
 0       24          17.0               17.0               0.56      0.33   
 1       25          17.5               17.5               0.56      0.33   
 2       26          18.0               18.0               0.56      0.33   
 3       27          18.5               18.5               0.56      0.33   
 
    prices_a  
 0     18.66  
 1     19.16  
 2     19.66  
 3     20.16  ,
    WTP (v)  phi (passive)  hospital_profit  insurer_profits  shares  prices
 0       24          11.25            11.25             0.56    0.33   12.91
 1       25          11.25            11.25             0.56    0.33   12.91
 2       26          11.25            11.25             0.56    0.33   12.91
 3       27          11.25            11.25             0.56    0.33   12.91,
    COST (λ)  phi (active)  hospital_profit_a  insurer_profits_a  shares_a  \
 0         4          16.5               16.5               0.44      0.3